## Modeling Notebook

#### Load request modules

In [432]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import root_mean_squared_error

from tensorflow.keras import Sequential
from tensorflow.keras.layers import MaxPooling2D, Conv2D, Flatten, Dense, InputLayer

#### Load data

In [103]:
raw_env_air_df = pd.read_parquet('../data/processed/air_quality_hotspot_data-20241101_20241130.parquet')

#### Merge environment and satellite hotspot data

In [121]:
# limit date range
env_air_df = raw_env_air_df.reset_index().copy()
env_air_df = env_air_df[
(env_air_df['date']>='2024-11-26 00:00:00+00:00') & \
(env_air_df['date']<='2024-11-30 23:59:59+00:00')].reset_index().copy() #.set_index(['date','latitude','longitude'])

In [395]:
# set coordinate degree step
def downsamping(env_air_df, loc_step=0.5):
    df = env_air_df.copy().drop(columns=['index'])
    round_div = 1 / loc_step
    
    # downsampling coordinate
    df['latitude'] = (df['latitude'] * round_div).round() / round_div
    df['longitude'] = (df['longitude'] * round_div).round() / round_div
    res = df.groupby(['date','latitude','longitude']).mean()
    
    return res

In [82]:
# set momdel variables
def split_data(avg_env_air_df):
    X = avg_env_air_df[[c for c in avg_env_air_df.columns if c not in ['pm10','pm2_5','carbon_monoxide','nitrogen_dioxide','sulphur_dioxide','ozone','dust']]]
    y = avg_env_air_df[['pm2_5']]
    X.shape, y.shape
    
    # # split data for train and test
    return train_test_split(X, y, train_size=0.8, random_state=42)

#### Defined models

In [442]:
def run_model(models, X_train, X_test, y_train, y_test):
    scores = []
    
    for i, model in enumerate(models):
        if 'Forest' in model['name']:
            y_train = y_train.values.ravel()
            y_test = y_test.values.ravel()
        grid = GridSearchCV(model['pipeline'], param_grid=model['param'], n_jobs=8)
        grid.fit(X_train, y_train)
        models[i]['fit_model'] = grid.best_estimator_.fit(X_train, y_train)
        models[i]['model'] = grid.best_estimator_
        print(f'Running model: {i}. {model["name"]}')
        preds = grid.predict(X_test)
        #plot_pred(y_test, preds)
        scores.append({'name':model['name'],
                       'cv_score': grid.best_score_,
                       'train_score': grid.best_estimator_.score(X_train, y_train),
                       'test_score': grid.best_estimator_.score(X_test, y_test),
                       'test_rmse': root_mean_squared_error(y_test, preds),
                      })
    print('-'*70)
    score_df = pd.DataFrame(scores).set_index('name')
    return models, score_df

---

In [427]:
# plot test vs. predict
def plot_pred(y_test, preds):
    sns.scatterplot(x=pd.Series(y_test.values[:,-1]), y=pd.Series(preds));
    plt.title('Test vs. Predict')
    plt.xlabel('Test')
    plt.ylabel('Predict')
    plt.show()

In [444]:
avg_env_air_df = downsamping(env_air_df, loc_step=0.5)
X_train, X_test, y_train, y_test = split_data(avg_env_air_df)

#avg_env_air_df.head(5)

models = [
    {'name': 'Linear Regression',
     'pipeline': Pipeline([
                ('sc', StandardScaler()),
                ('lr', LinearRegression())
              ]), 
     'param': {
     }
    },
    {'name': 'XGBoost',
     'pipeline': Pipeline([
                ('sc', StandardScaler()),
                ('xgb', xgb.XGBRFRegressor())
             ]), 
     'param': {'xgb__learning_rate': [1]}
    },
    {'name': 'ElasticNet',
     'pipeline': Pipeline([
                ('sc', StandardScaler()),
                ('esn', ElasticNet())
             ]), 
     'param': {
         'esn__alpha': [0.1, 0.5, 1],
         'esn__max_iter': [10, 100, 500, 1000],
     }
    },
    {'name': 'Random Forest',
     'pipeline': Pipeline([
                ('sc', StandardScaler()),
                ('rf', RandomForestRegressor())
             ]), 
     'param': {
     }
    },
]

models, scores = run_model(models, X_train, X_test, y_train, y_test)

scores.sort_values(by='cv_score', ascending=False)

Running model: 0. Linear Regression
Running model: 1. XGBoost
Running model: 2. ElasticNet
Running model: 3. Random Forest
----------------------------------------------------------------------


,cv_score,train_score,test_score,test_rmse
name,,,,
Random Forest,0.794805,0.973076,0.802088,2.841620
XGBoost,0.570976,0.589400,0.569322,4.191858
Linear Regression,0.425060,0.425865,0.419179,4.868004
ElasticNet,0.409982,0.410472,0.404714,4.928249
